In [1]:
import os
import json
import vtk 
import slicer   
import MultiVolumeImporter  
#from ExtractCenterline import ExtractCenterlineLogic  
import ExtractCenterline 

############################################################################
############################################################################

### SETUP/IMPORT ### 

print("Loading file for centerline extraction...")

# LOAD FILE AS VOLUME (CURRENTLY SAME AS SEGMENTATION) 
volume = slicer.util.loadVolume("J:\\SOHEIL_PROJECT\\SEGMENTATION\\MESAL-7012705\\Segmentation_final_v2-5_use-label.nrrd")

# LOAD IN A SEGMENTATION FILE 
# USED TO SPECIFY THAT FUTURE PROCESSES WILL INTERACT WITH THIS FILE (NOTE FUTURE USES OF 'vessel_seg') 
vessel_seg = slicer.util.loadSegmentation("J:\\SOHEIL_PROJECT\\SEGMENTATION\\MESAL-7012705\\Segmentation_final_v2-5_use-label.nrrd")

segmentation = vessel_seg.GetSegmentation() 

# OPTION TO APPLY MODIFIER TO LOADED FILE (SMOOTHING EXAMPLE) 
segmentation.SetConversionParameter("Smoothing factor", "0.1") 

# OPTION TO CREATE THE 3D VIEW 
vessel_seg.CreateClosedSurfaceRepresentation() 

# ASSIGNS SPECIFIC SEGMENTATION LABEL ('Segment_1') TO VARIABLE 
segmentID = segmentation.GetSegmentIdBySegmentName('Segment_1') 

# CREATES AN EMPTY 'vtkPolyData' OBJECT 
segmentVtkPolyData = vtk.vtkPolyData() 

# GET 'vtkPolyData' REPRESENTATION OF SEGMENT SURFACE INTO 'segmentVtkPolyData' VARIABLE 
    # BY PASSING IN SEGMENTATION LABEL & 'vtkPolyData' VARIABLES 
vessel_seg.GetClosedSurfaceRepresentation(segmentID, segmentVtkPolyData) 

print(segmentID + " loaded for centerline extraction...") 


Loading file for centerline extraction...
Segment_1 loaded for centerline extraction...


In [2]:
############################################################################
############################################################################

### PREPROCESSING ### 
# USED TO SHORTN PATHS TO FUNCTIONS USED IN MODULE 
extractLogic = ExtractCenterline.ExtractCenterlineLogic() 

## PREPROCESS SEGMENTATION LABLEL ('Segment_1' IN THIS CASE) 
# USED TO INITIALIZE 'EXTRACT CENTERLINE' PARAMETERS 
inputSurfacePolyData = extractLogic.polyDataFromNode(vessel_seg, segmentID) 

# CAN BE MODIFIED AS NEEDED 
# SEE 'ExtractCenterline.py' FOR DEFAULT SETTINGS 
preprocessEnabled = True  # (self._parameterNode.GetParameter("PreprocessInputSurface") == "true") 
targetNumberOfPoints = 15000.0  # float(self._parameterNode.GetParameter("TargetNumberOfPoints")) 
decimationAggressiveness = 4.0 #float(self._parameterNode.GetParameter("DecimationAggressiveness")) 
subdivideInputSurface = False  # (self._parameterNode.GetParameter("SubdivideInputSurface") == "true") 
autoDetectEndPointsPushButton = True # (self._parameterNode.GetParameter(autoDetectEndPointsPushButton") == "true") 

slicer.util.showStatusMessage( 
    "Preprocessing surface before centerline extraction..." 
) 

print("Preprocessing surface before centerline extraction...") 

slicer.app.processEvents() 

# PASSESES IN THE ABOVE PREPROCESS VALUES 
# WILL BE USED DURING CENTERLINE EXTRACTION 
preprocessedPolyData = extractLogic.preprocess( 
    inputSurfacePolyData, 
    targetNumberOfPoints, 
    decimationAggressiveness, 
    subdivideInputSurface, 
) 
  
print("Preprocessing complete...")

Preprocessing surface before centerline extraction...
Preprocessing complete...


In [21]:
############################################################################
############################################################################

### EXTRACT CENTERLINE ### 

centerlineCurveNode = None 

print("Auto-detecting surface model endpoints...") 

# ATTEMPTS TO 'CLICK' THE 'AUTO-DETECT' BUTTON AND STORE THE END POINT MARKUPS IN THE 'endPointsMarkupsNode' VARIABLE 
# Auto-detect the endpoints 
endPointsMarkupsNode = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLMarkupsFiducialNode", "Centerline endpoints") 
networkPolyData = extractLogic.extractNetwork(preprocessedPolyData, endPointsMarkupsNode) 
startPointPosition=None 
endpointPositions = extractLogic.getEndPoints(networkPolyData, startPointPosition) 
endPointsMarkupsNode.RemoveAllMarkups() 

for position in endpointPositions: 
  endPointsMarkupsNode.AddControlPoint(vtk.vtkVector3d(position)) 

print("End points autodetected...")

# USED TO STORE THE CENTERLINE CURVE
if centerlineCurveNode is None: 
    centerlineCurveNode = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLMarkupsCurveNode", "Centerline curve") 

slicer.util.showStatusMessage("Extracting centerline...")  

# FORCES UPDATE TO ENSURE MODIFIED SETTINGS ARE APPLIED BEFORE RUNNING THE MODULE 
slicer.app.processEvents()  # force update 

# PASSES CENTERLINE/DIAGRAM DATA TO 2 SEPARATE VARIABLES 
# 'preprocessedPolyData'PREPROCESS MODIFIERS, MARKUPS/ENDPOINTS NODE PASSED INTO 'extractCenterline' 
centerlinePolyData, voronoiDiagramPolyData = extractLogic.extractCenterline( 
        preprocessedPolyData, endPointsMarkupsNode, curveSamplingDistance=0.001) #'end_pts' replaced with 'endPointsMarkupsNode' 

slicer.util.showStatusMessage("Generating curves and quantification results table...") 

slicer.app.processEvents()  # force update  

centerlinePropertiesTableNode = None  

print("Extracting centerline curve tree...")

# USED TO GENERATE CENTERLINE SEGMENTS FROM DATA
extractLogic.createCurveTreeFromCenterline( 
    centerlinePolyData, centerlineCurveNode, centerlinePropertiesTableNode 
)

if centerlineCurveNode.GetNumberOfControlPoints() == 2: 
    # If Extraction had an error, likely due to too high decimation aggressiveness 
    # Try again 
    slicer.util.errorDisplay( 
        "Centerline generation failed, possibly due to decimationAggressiveness being too high." 
    )

print("Segmented centerline curve tree created...")

Auto-detecting surface model endpoints...
End points autodetected...
Extracting centerline curve tree...


<class 'AttributeError'>: 'MRMLCorePython.vtkMRMLModelNode' object has no attribute 'SetControlPointPositionsWorld'

In [ ]:
# COPIED FROM CHATGPT TO CONVERT MARKUP CURVE (NODE) TO MODEL (NODE) TO EXTACT CENTERLINE CURVE SEGMENTS TO .vtk
# NEEDS TO BE EDITED/TESTED

import slicer

# Load the markup file containing the curves
markupNode = slicer.util.loadMarkupsFiducialList("path_to_markup_file")

# Iterate through each markup curve
for i in range(markupNode.GetNumberOfMarkups()):
    curveMarkup = markupNode.GetMarkupPointVector(i)
    
    # Create a vtkPoints object to hold the curve points
    points = vtk.vtkPoints()
    
    # Add each point of the curve to the vtkPoints object
    for j in range(curveMarkup.GetNumberOfPoints()):
        point = [0, 0, 0]
        curveMarkup.GetNthPoint(j, point)
        points.InsertNextPoint(point)
    
    # Create a vtkCellArray to hold the curve vertices
    lines = vtk.vtkCellArray()
    
    # Add the curve vertices to the vtkCellArray
    line = vtk.vtkPolyLine()
    line.GetPointIds().SetNumberOfIds(points.GetNumberOfPoints())
    for j in range(points.GetNumberOfPoints()):
        line.GetPointIds().SetId(j, j)
    lines.InsertNextCell(line)
    
    # Create a vtkPolyData to hold the curve geometry
    polyData = vtk.vtkPolyData()
    polyData.SetPoints(points)
    polyData.SetLines(lines)
    
    # Create a model node and set its polydata
    modelNode = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLModelNode")
    modelNode.SetAndObservePolyData(polyData)
    modelNode.SetName("Curve_" + str(i))
    
    # Optionally, adjust parameters such as color, thickness, etc., for visualization
    displayNode = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLModelDisplayNode")
    displayNode.SetColor(1, 0, 0)  # Red color
    displayNode.SetLineWidth(2.0)   # Thickness
    modelNode.SetAndObserveDisplayNodeID(displayNode.GetID())

# Optional: remove the markup node if it is no longer needed
# slicer.mrmlScene.RemoveNode(markupNode)


In [8]:
### TESTING ###

# print("Saving Centerline Curve and Centerline PolyData to files...") 

# slicer.mrmlScene.GetNodeByID(centerlineCurveNode.GetID()).GetName() 

# slicer.util.exportNode(centerlineCurveNode, "J:\\SOHEIL_PROJECT\\SEGMENTATION\\MESAL-7012705\\Segmentation_final_v2-5_use-label\\S-Final-2-label_extractedCenterlineCurve.nrrd") 
# #slicer.util.exportNode(centerlineCurveNode, "C:\\Users\\Adam\\Desktop\\Programs\\code_testing\\python_file_rename_copy\\Case-22-0.5_A_80KV_Qr40d_3_53062428\\MarkUps\\88-Jensen\\S-Final-2-label_extractedCenterlineCurve.nrrd") 
# #slicer.util.exportNode(centerlinePolyData, "C:\\Users\\Adam\\Desktop\\Programs\\code_testing\\python_file_rename_copy\\Case-22-0.5_A_80KV_Qr40d_3_53062428\\MarkUps\\88-Jensen\\S-Final-2-label_extractedCenterlinePolyData.nrrd") 


# #curveNode = slicer.mrmlScene.GetSubjectHierarchyNode() 
# #exportFolderItemId = curveNode.CreateFolderItem(curveNode.GetSceneItemID(), "Segments") 
# #slicer.modules.segmentations.logic().ExportAllSegmentsToModels(curveNode, exportFolderItemId) 

# print("Files save\nProcess Finished") 

#segmentation = aorta_seg.GetSegmentation()
#segmentID = segmentation.GetSegmentIDBySegmentName('Segment_1')
vessel_seg.CreateClosedSurfaceRepresentation() # just to ensure this representation exists
segmentVtkPolyData = vtk.vtkPolyData() # create empty vtkPolyData object
# Get vtkPolyData representation of segment surface into segmentVtkPolyData variable
vessel_seg.GetClosedSurfaceRepresentation(segmentID, segmentVtkPolyData)






True

In [20]:
### TESTING ###


#curveData = slicer.util.arrayFromMarkupsCurveData(centerlineCurveNode, "Curvature", world=True)

#print(curveData)

print("Storing curvature data...")

#curveNode = slicer.util.getNode('Centerline*')

curveNode = slicer.util.getNode('Centerline curve*')
#print(curveNode)

# GETS THE CURVATURE MEAN FOR EACH SEGMENT & ENABLES CURVATURE COMPUTATION
curveNode.GetMeasurement('curvature mean').SetEnabled(True)
#print(curveNode)

# RETURNS INTERPOLATED CURVE POINT POSITIONS OF A MARKUP NODE AS ROWS IN A NUMPY ARRAY (SIZE Nx3)
# SETTING world=True ALLOWS POINT COORDINATES TO BE RETURNED IN THE WORLD COORDINATE SYSTEM
# https://slicer.readthedocs.io/en/latest/developer_guide/slicer.html#slicer.util.arrayFromMarkupsCurveData
curvature = slicer.util.arrayFromMarkupsCurveData(curveNode, "Curvature", world=True)

print(type(curvature))

#curveNodeVol = slicer.mrmlScene.AddNewNodeByClass("vktMRMLScalarVolumeNode")
curveNodeVol = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLModelNode", )
# inputSurfaceModelNode = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLModelNode", "tempInputSurfaceModel")
# INITIALIZE EMPTY VOLUME NODE
#vtk.util.numpy_support.numpy_to_vtk(curvature)



curveNodeModel = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLModelNode", "Centerline curves model")

# COPY THE CURVATURE DATA TO THE INITIALIZED EMPTY VOLUME NODE
#slicer.util.updateVolumeFromArray(curveNodeVol, curvature)

print(type(curvature))

print(type(curveNodeVol))

print(type(curveNode))

slicer.util.saveNode(curveNode, "J:\\SOHEIL_PROJECT\\SEGMENTATION\\MESAL-7012705\\Segmentation_final_v2-5_use-label\\complete_centerline_curve_segments.vtk")


#print(curvature)
#curveData = slicer.util.arrayFromMarkupsCurveData(centerlineCurveNode, "Centerline*", world=True)
#curveData = slicer.util.arrayFromMarkupsCurveData(centerlineCurveNode, "PedigreeIDs", world=True)

#print(curvature)

#print(curveData)

#=============================================================================================
#=============================================================================================

# TEST CODE TO OUTPUT SEGMENT AVERAGE CURVATURE DATA

# outputFileName = "J:\\SOHEIL_PROJECT\\SEGMENTATION\\MESAL-7012705\\meanCurvature.json"

# vtk.util.numpy_support.vtk_to_numpy(curvature)
# print(type(curvature))

# vol_node = slicer.mrmlScene.AddNewNodeByClass("vktMRMLScalarVolumeNode")
# print(type(vol_node))
# slicer.util.updateVolumeFromArray(vol_node,curvature)

# myStorageNode = curvature.CreateDefaultStorageNode()
# myStorageNode.SetFileName("J:\\SOHEIL_PROJECT\\SEGMENTATION\\MESAL-7012705\\meanCurvature.vtk")
# myStorageNode.WriteData(vol_node)

#=============================================================================================
#=============================================================================================

# EXPORT MEAN CURVATURE DATA TO JSON FILE
#with open(outputFileName, 'w') as outfile:
#  json.dump(curvature, outfile)


#slicer.util.plot(curvature)

print("Curvature data stored...")

Storing curvature data...
<class 'numpy.ndarray'>


<class 'AttributeError'>: module 'slicer.util' has no attribute 'updateModelFromArray'

In [4]:
############################################################################
############################################################################

### GET 1-SEGMENT CENTERLINE MODEL ###

print("Extracting centerline model...")

# INSTANTIATE MODEL NODE
centerlineModel = slicer.vtkMRMLModelNode()
# APPLY 'centerlinePolyData'TO NEW MODEL NODE
centerlineModel.SetAndObservePolyData(centerlinePolyData)
# USED TO EDIT CENTERLINE MODEL APPEARANCE
centerlineModelDisplay = slicer.vtkMRMLModelDisplayNode()

# USED TO SET APPEARANCE OF MODEL NODE
centerlineModelDisplay.SetColor(1, 1, 0)
centerlineModelDisplay.BackfaceCullingOff()
centerlineModelDisplay.SetOpacity(0.8)
centerlineModelDisplay.SetPointSize(3)

centerlineModelDisplay.SetSliceIntersectionVisibility(True)
centerlineModelDisplay.SetVisibility(True)

# ADDS MODEL NODE APPEARANCE INFO TO SLICER SCENE
slicer.mrmlScene.AddNode(centerlineModelDisplay)
centerlineModel.SetAndObserveDisplayNodeID(centerlineModelDisplay.GetID())
centerlineModelDisplay.SetInputPolyDataConnection(centerlineModel.GetPolyDataConnection())
# ADDS CENTERLINE MODEL NODE TO SLICER SCENE
slicer.mrmlScene.AddNode(centerlineModel)

print("Centerline model extracted")

print("Centerline extraction complete")

Extracting centerline model...
Centerline model extracted
Centerline extraction complete


In [5]:
############################################################################
############################################################################

### EXPORT RESULTS TO FILES ###

print("Exporting 1-segment centerline model to file...")

# EXPORT 1-SEGMENT CENTERLINE MODEL TO .vtk FILE
slicer.util.saveNode(centerlineModel, "J:\\SOHEIL_PROJECT\\SEGMENTATION\\MESAL-7012705\\Segmentation_final_v2-5_use-label\\complete_centerline_model.vtk")

print("1-segment centerline model exported...")

print("Exporting centerline curvature segments to file...")

True